In [ ]:
#default_exp lightcurve
%load_ext autoreload
from nbdev.showdoc import show_doc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Light Curve
> Generate light curves from cells

In [ ]:
#export
import numpy as np
import pylab as plt
import pandas as pd


In [ ]:
#export
from light_curves.loglike import (LogLike, GaussianRep, Gaussian2dRep, PoissonRep, PoissonRepTable)

class _LightCurve(object):
    """ Apply likelihood fits to a set of cells
    
    """

    rep_class =  [GaussianRep, Gaussian2dRep, PoissonRep, PoissonRepTable]
    rep_list =   'gauss gauss2d poisson poissontable'.split()

    def __init__(self, config, 
                all_cells, 
                source, 
                min_exp:  'minimum exposure factor'= 0.3,
                rep_name: 'likelihood rep name'='',
    
                ):
        """Apply fits to the likelihoods for a set of cells
        
        parameters:
           - cells : a table with index t, columns  tw, n, fexp, w, S, B
           - min_exp : minimum fractional exposure allowed
           - rep_name : represention to use 
        
        Generates a DataTable with columns n, fexp, fit
        
        """

        self.source_name = source.name
 

        # select the set of cells
        cells = all_cells.query(f'fexp>{min_exp}').copy()
        
        # generate a list of LogLike objects for each
        cells.loc[:,'loglike'] = cells.apply(LogLike, axis=1)
        if config.verbose>0:
            print(f'Loaded {len(cells)} / {len(all_cells)} cells with exposure >'\
                  f' {min_exp} for light curve analysis')

        # analyze using selected rep
        rep_name = rep_name or config.likelihood_rep
        
        if rep_name not in self.rep_list:
            raise Exception(f'Unrecognized rep: "{rep_name}", must be one of {self.rep_list}')
        repcl = self.rep_class[self.rep_list.index(rep_name)]

        if config.verbose>1:
            print(f'Fitting likelihoods with {rep_name} representation')
        
        # making output with reduced columns 
        self.ll_fits = cells['n fexp'.split()].copy()
        self.ll_fits.loc[:,'fit'] = cells.loglike.apply(repcl)

    def __repr__(self):
        return f'{self.__class__.__name__}: source "{self.source_name}" fit with {len(self.ll_fits)} cells'


    @property
    def dataframe(self):
        """return the DataFrame
        """
        return self.ll_fits

In [ ]:
#export    

from light_curves.config import Config, Files, PointSource
from light_curves.cells import get_cells

def get_lightcurve(config, files, source):
    """
    
    """
    fcache = files.cache/f'{source.filename}_lightcurve.pkl' if config.use_cache else None

    if fcache and fcache.exists():    
        if config.verbose>1:
            print(f'restoring the light curve from {fcache} ' , end='')
        lc = pd.read_pickle(fcache)

        return lc
    
    all_cells = get_cells(config, files, source)
    lc = _LightCurve(config, all_cells, source).dataframe
    
    if fcache:
        if config.verbose>1:
            print(f'Saveinga light curve at {fcache} ')
        lc.to_pickle(fcache)
    
    return lc
    

In [ ]:

from light_curves.config import Config, Files, PointSource
from light_curves.cells import get_cells

config = Config()
files = Files()
source = PointSource('Geminga')

if files.valid:
    print(f'Loading cells with source {source.name} ')
    lc = get_lightcurve(config, files, source)
else:
    print('Not testing since no files.')
    lc = None
if lc is not None:
    print(f'\nFirst cell fit: {lc.iloc[0]["fit"]}')

Loading cells with source Geminga 
restoring the light curve from /tmp/light_curves/Geminga_lightcurve.pkl 
First cell fit: light_curves.loglike.PoissonRep: flux: 0.927[1+0.076-0.079], limit: 1.05, ts: 481.9


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_load_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 07_cells.ipynb.
Converted 09_poisson.ipynb.
Converted 10_loglike.ipynb.
Converted 11_lightcurve.ipynb.
Converted index.ipynb.
Mon Dec  7 16:00:44 PST 2020


In [ ]:
#export
def flux_plot(config, lightcurve, ts_max=9, xerr=0.5, title=None, ax=None, **kwargs): 
    """Make a plot of flux with according to the representation
    """
    kw=dict(yscale='linear',xlabel='MJD', ylabel='relative flux',)
    kw.update(**kwargs)
    df=lightcurve
    if config.likelihood_rep=='poisson':
        ts = df.ts
        limit = ts<ts_max
        bar = df.loc[~limit,:]
        lim = df.loc[limit,:]
    else: 
        bar=df; lim=[]

    fig, ax = plt.subplots(figsize=(12,4)) if ax is None else (ax.figure, ax)\
        if ax is not None else (ax.figure,ax)

    # the points with error bars
    t = bar.t
    xerr = bar.tw/2
    y =  bar.flux.values
    if config.likelihood_rep=='poisson':
        dy = [bar.errors.apply(lambda x: x[i]).clip(0,4) for i in range(2)]
    elif self.rep=='gauss' or self.rep=='gauss2d':
        dy = bar.sig_flux.clip(0,4)
    else: assert False     
    ax.errorbar(x=t, y=y, xerr=xerr, yerr=dy, fmt=' ', color='silver')

    # now do the limits
    if len(lim)>0:
        t = lim.t
        xerr = lim.tw/2
        y = lim.limit.values
        yerr=0.2*(1 if kw['yscale']=='linear' else y)
        ax.errorbar(x=t, y=y, xerr=xerr,
                yerr=yerr,  color='lightsalmon', 
                uplims=True, ls='', lw=2, capsize=4, capthick=0,
                alpha=0.5)

    #ax.axhline(1., color='grey')
    ax.set(**kw)
    ax.set_title(title or f'{self.source_name}, rep {self.rep}')
    ax.grid(alpha=0.5)

In [ ]:
# ?lux_plot(config, lc)